In [ ]:
from pystac_client import Client
from odc.stac import load

from planetary_computer import sign_url
from ipyleaflet import basemaps, basemap_to_tiles

import numpy as np
from odc.ui import select_on_a_map

import warnings
import folium

In [ ]:
# Microsoft Planetary Computer STAC Catalog URL
catalog = "https://planetarycomputer.microsoft.com/api/stac/v1"

# Create a STAC Client
client = Client.open(catalog)

# Parameters for the query
collection = "sentinel-1-rtc"
datetime = "2024"

In [ ]:
# Set study area name for outputs
study_area = "se-asia"

# Plot interactive map to select area
basemap = basemap_to_tiles(basemaps.Esri.WorldImagery)
geom = select_on_a_map(height="600px", layers=(basemap,), center=(12, 120), zoom=4)
geom

In [ ]:
items = client.search(
    collections=["sentinel-1-rtc"],
    intersects=geom,
    datetime=datetime
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(
    items,
    geopolygon=geom,
    measurements=["vv", "vh"],
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
    patch_url=sign_url,
)

# Convert to decibels
data["vv_db"] = 10 * np.log10(data.vv)
data["vh_db"] = 10 * np.log10(data.vh)

data

In [ ]:
# Supress the RuntimeWarning for log10(0)
warnings.filterwarnings("ignore", category=RuntimeWarning)

vv_max = data.vv_db.max(dim=["time"]).compute()
vh_max = data.vh_db.max(dim=["time"]).compute()

In [ ]:
m = folium.Map()

vv_max.odc.add_to(m, cmap="viridis", name="VV max")
vh_max.odc.add_to(m, cmap="viridis", name="VH max")

folium.LayerControl().add_to(m)
m.fit_bounds(vv_max.odc.map_bounds())
m